In [3]:
#Importing the necessary libraries for our Deep Learning Model

import pandas as pd
import numpy as np 
import tensorflow as tf 
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Activation, Dropout, Flatten, BatchNormalization
from keras.preprocessing import image
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGeneratora
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

ImportError: cannot import name 'ImageDataGeneratora' from 'keras.preprocessing.image' (C:\Users\amitn\AppData\Roaming\Python\Python38\site-packages\keras\preprocessing\image.py)

In [4]:
train_df = pd.read_csv("train.csv")
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_df = pd.read_csv("test.csv")
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x_train = train_df.drop(['label'], axis = 1)
y_train = train_df['label']
del train_df

In [7]:
x_train = x_train/255.0
test_df = test_df/255.0

x_train = x_train.values.reshape(-1,28,28,1)
test_df = test_df.values.reshape(-1,28,28,1)


y_train = to_categorical(y_train, num_classes=10)

In [8]:

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=2)

In [9]:
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

learning_rate_reduction2 = ReduceLROnPlateau(monitor='val_categorical_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [10]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [11]:
def cust2() : 
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28,1)))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten()) 
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer = optimizer,
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

cust2_model = cust2()
cust2_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       5

In [12]:
# Fit the model
from keras.callbacks import ModelCheckpoint
path_model='custcnn.h5' 
cust_history = cust2_model.fit(datagen.flow(X_train,Y_train, batch_size=64),
                              epochs = 30, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // 64,
                              callbacks=[learning_rate_reduction,ModelCheckpoint(filepath=path_model)])

Epoch 1/30
590/590 - 36s - loss: 0.3335 - accuracy: 0.8994 - val_loss: 0.1968 - val_accuracy: 0.9602
Epoch 2/30
590/590 - 20s - loss: 0.0985 - accuracy: 0.9718 - val_loss: 0.1044 - val_accuracy: 0.9693
Epoch 3/30
590/590 - 21s - loss: 0.0757 - accuracy: 0.9788 - val_loss: 0.0448 - val_accuracy: 0.9883
Epoch 4/30
590/590 - 22s - loss: 0.0640 - accuracy: 0.9815 - val_loss: 0.0418 - val_accuracy: 0.9895
Epoch 5/30
590/590 - 22s - loss: 0.0598 - accuracy: 0.9829 - val_loss: 0.0329 - val_accuracy: 0.9900
Epoch 6/30
590/590 - 23s - loss: 0.0528 - accuracy: 0.9849 - val_loss: 0.0395 - val_accuracy: 0.9898
Epoch 7/30
590/590 - 21s - loss: 0.0454 - accuracy: 0.9880 - val_loss: 0.0329 - val_accuracy: 0.9917
Epoch 8/30
590/590 - 22s - loss: 0.0457 - accuracy: 0.9879 - val_loss: 0.0390 - val_accuracy: 0.9910
Epoch 9/30
590/590 - 22s - loss: 0.0430 - accuracy: 0.9881 - val_loss: 0.0311 - val_accuracy: 0.9926
Epoch 10/30
590/590 - 22s - loss: 0.0404 - accuracy: 0.9890 - val_loss: 0.0244 - val_accura

In [13]:
from keras.models import model_from_json

model_json = cust2_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

print("json file saved")

json file saved


In [14]:
cust2_model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [11]:
# from  tensorflow.keras.layers import MaxPool2D, Input
# from tensorflow.keras.optimizers import Adam
# from keras.models import Model
# DROPOUT_RATE = 0.3
# CONV_ACTIVATION = "relu"

# img_in = Input(shape=(28,28,1))

# X = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(img_in)
# X = BatchNormalization()(X)
# X = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)

# X = MaxPool2D((2, 2), strides=(2, 2), padding='same')(X)
# X = Dropout(DROPOUT_RATE)(X)


# X = Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)

# X = MaxPool2D((2, 2), strides=(2, 2), padding='same')(X)
# X = Dropout(DROPOUT_RATE)(X)

# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)

# X = MaxPool2D((2, 2), strides=(2, 2), padding='same')(X)
# X = Dropout(DROPOUT_RATE)(X)

# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = MaxPool2D((2, 2), strides=(2, 2), padding='same')(X)
# X = Dropout(DROPOUT_RATE)(X)

# X = Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation=CONV_ACTIVATION)(X)
# X = BatchNormalization()(X)
# X = MaxPool2D((2, 2), strides=(2, 2), padding='same')(X)
# X = Dropout(DROPOUT_RATE)(X)

# X = Flatten()(X)
# X = Dense(2048, activation="relu")(X)
# X = Dropout(DROPOUT_RATE)(X)
# X = Dense(1024, activation="relu")(X)
# X = Dropout(DROPOUT_RATE)(X)
# X = Dense(512, activation="relu")(X)
# X = Dropout(DROPOUT_RATE)(X)

# out = Dense(10, activation='softmax')(X)

# vgg_model = Model(inputs=img_in, outputs=out)
# vgg_model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["categorical_accuracy"])

In [12]:
# vgg_history = vgg_model.fit(datagen.flow(X_train,Y_train, batch_size=32),
#                               epochs = 23, validation_data = (X_val,Y_val),
#                               verbose = 2, steps_per_epoch=X_train.shape[0] // 32,
#                               callbacks=[learning_rate_reduction2])

Epoch 1/23
1181/1181 - 95s - loss: 1.1693 - categorical_accuracy: 0.5937 - val_loss: 0.8051 - val_categorical_accuracy: 0.8736
Epoch 2/23
1181/1181 - 81s - loss: 0.5052 - categorical_accuracy: 0.9056 - val_loss: 0.2175 - val_categorical_accuracy: 0.9648
Epoch 3/23
1181/1181 - 81s - loss: 0.4709 - categorical_accuracy: 0.9226 - val_loss: 0.1188 - val_categorical_accuracy: 0.9743
Epoch 4/23
1181/1181 - 81s - loss: 0.4403 - categorical_accuracy: 0.9291 - val_loss: 0.1190 - val_categorical_accuracy: 0.9814
Epoch 5/23
1181/1181 - 80s - loss: 0.4192 - categorical_accuracy: 0.9335 - val_loss: 0.4550 - val_categorical_accuracy: 0.9662
Epoch 6/23
1181/1181 - 79s - loss: 0.4958 - categorical_accuracy: 0.9336 - val_loss: 0.1203 - val_categorical_accuracy: 0.9764
Epoch 7/23
1181/1181 - 80s - loss: 0.4794 - categorical_accuracy: 0.9352 - val_loss: 0.1540 - val_categorical_accuracy: 0.9569

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 8/23
1181/1181 - 79s - l